In [18]:
from datetime import datetime
import hdbscan
from pathlib import Path
from typing import Optional
import pandas as pd
from loguru import logger
from sklearn.datasets import make_blobs


In [ ]:
# Añado clúster a la inferencia de Llava
llava_classification = pd.read_csv()
# Mergeamos con el resultado de etiquetado manual
manual_classification = pd.read_csv("data/results_manual_annotation/inference_results.csv", sep=";", header=0)
manual_classification['manual_category'] = manual_classification['manual_category'].apply(lambda x: x.upper())
# Unimos ambos por imagen
result = llava_classification.merge(manual_classification[["img","manual_category"]],on="img",how="left")

# # Hay una que se queda NA, creo que es por algo del formato:
# # Rellenamos con roads que es lo que Carlos ha puesto, pero no lo coge
result['manual_category'] = result['manual_category'].astype(str).fillna("ROADS")


print(result.head(10).to_string())
result.to_csv("df_final_lvlm_vs_manual.csv",sep=";")



                                       img          category_llava                               manual_category
0  aiguestortes_NA_1366_47701956991__b.jpg                 SHELTER                                     LANDFORMS
1  aiguestortes_NA_1633_31399087577__b.jpg                  PLANTS                       VEGETATION AND HABITATS
2  aiguestortes_NA_1633_50177463812__b.jpg  TERRESTRIAL ACTIVITIES                        TERRESTRIAL ACTIVITIES
3  aiguestortes_NA_1690_52491627395__b.jpg                  BRIDGE                             TRACKS AND TRAILS
4  aiguestortes_NA_2008_19242320949__b.jpg                  PLANTS                                        PLANTS
5  aiguestortes_NA_4531_49900773177__b.jpg                 ANIMALS                                       ANIMALS
6  aiguestortes_NA_4792_24046167316__b.jpg    HERITAGE AND CULTURE                                  NOT RELEVANT
7    guadarrama_NA_1089_20943542884__b.jpg                  CITIES                            TO

In [50]:
nan_rows = result[result.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [img, category_llava, manual_category]
Index: []


In [51]:
import pandas as pd
import matplotlib.pyplot as plt

# Función para filtrar y graficar
def plot_line_chart(df, metric_name):
    df = df.fillna(0)  # Reemplaza NaN con 0
    df = df.loc[(df != 0).any(axis=1)]  # Filtrar filas donde al menos un valor es distinto de 0

    if df.empty:
        print("No hay datos para graficar después del filtrado.")
        return

    # Transponer el DataFrame para que los umbrales estén en el eje X
    df_t = df.T

    # Crear la figura
    plt.figure(figsize=(12, 6))

    # Graficar cada categoría con líneas
    for category in df_t.columns:
        plt.plot(df_t.index, df_t[category], marker='o', linestyle='-', label=category, alpha=0.7)

    # Configurar etiquetas y título
    plt.xlabel('Umbral de la métrica')
    plt.ylabel(metric_name)
    plt.title(f'Evolución de {metric_name} por Categoría')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Leyenda fuera de la gráfica
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()


In [54]:
from sklearn.metrics import classification_report

# Generar el reporte de clasificación
report = classification_report(
    result['manual_category'], 
    result['category_llava'], 
    output_dict=True, 
    zero_division=0
)

# Convertir a DataFrame y transponer para que las categorías sean el índice
df_report = pd.DataFrame(report).T

# Mostrar el DataFrame con categorías a la izquierda y métricas como columnas
df_report = df_report.drop(columns=["support"], errors="ignore")
df_report = df_report.drop(index=["accuracy","macro avg","weighted avg","nan"], errors="ignore")
df_report = df_report.dropna(axis=0, how="all")
print(df_report)

os.makedirs("results_report",exist_ok=True)
df_report.to_csv("results_report/lvlm_vs_manual.csv", sep=";")


                                              precision    recall  f1-score
ACCOMMODATION                                  0.000000  0.000000  0.000000
AIR ACTIVITIES                                 0.607143  0.680000  0.641509
ANIMALS                                        0.937158  0.695740  0.798603
BAD_INFERENCE                                  0.000000  0.000000  0.000000
BRIDGE                                         0.622449  0.863208  0.723320
CITIES                                         0.291139  0.239583  0.262857
COMMERCE FACILITIES                            0.439024  0.247423  0.316484
DAM                                            0.791667  0.322034  0.457831
DOCK                                           0.000000  0.000000  0.000000
FUNGUS                                         1.000000  0.760000  0.863636
HERITAGE AND CULTURE                           0.152980  0.804938  0.257098
KNOWLEDGE                                      0.000000  0.000000  0.000000
LANDFORMS   